In [48]:
from google.colab import auth
import pandas as pd
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheet = gc.open('Malaysian Pay Gap: Data Submission (Responses) - Public').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# print(rows)

# Convert to a DataFrame and render.

df = pd.DataFrame.from_records(rows)

# Cleaning data and columns
cols = rows[0]
df.columns = cols
df = df.iloc[1:,:]
df.columns = ['Timestamp',
 'Age',
 'Race',
 'Gender identity',
 'Nationality',
 'What is your highest education qualification?',
 'Job title',
 'Years of experience',
 'How many jobs have you held in your career?',
 'Are you working in/from Malaysia?',
 'Is your job remote? ',
 'Where are you based?',
 'What type of company do you work for?',
 'Industry',
 "Industry_other",
 'Job specialisation',
 "Job specialisation_other",
 'Average working hours per day',
 'Average working days per week',
 'Monthly salary in Ringgit Malaysia (Gross: before EPF, tax deduction)',
 'Starting salary for your first job in Ringgit Malaysia',
 'How happy are you with your current salary/compensation package?',
 'How satisfied are you with your current job overall?',
 'Additional thoughts/insights you would like to share:']

# Change data types
numeric = ['Age', 'Years of experience', 'How many jobs have you held in your career?', 'Average working hours per day', 'Average working days per week', 'Monthly salary in Ringgit Malaysia (Gross: before EPF, tax deduction)',
           'Starting salary for your first job in Ringgit Malaysia', 'How happy are you with your current salary/compensation package?', 'How satisfied are you with your current job overall?']

st = ['Additional thoughts/insights you would like to share:']
times = ['Timestamp']

for x in numeric:
  df[x] = pd.to_numeric(df[x], 'coerce')

df['Timestamp'] = pd.to_datetime(df['Timestamp'], 'coerce')
print(df.head())

[['Timestamp', 'Age', 'Race', 'Gender identity', 'Nationality', 'What is your highest education qualification?', 'Job title', 'Years of experience', 'How many jobs have you held in your career?', 'Are you working in/from Malaysia?', 'Is your job remote? ', 'Where are you based?', 'What type of company do you work for?', 'Industry', "If selected 'other', please clarify:", 'Job specialisation', "If selected 'other', please clarify:", 'Average working hours per day', 'Average working days per week', 'Monthly salary in Ringgit Malaysia (Gross: before EPF, tax deduction)', 'Starting salary for your first job in Ringgit Malaysia', 'How happy are you with your current salary/compensation package?', 'How satisfied are you with your current job overall?', 'Additional thoughts/insights you would like to share:'], ['3/10/2022 12:51:52', '27', 'Chinese', 'Male', 'Malaysia', "Master's Degree (Overseas)", 'Data Analyst', '2', '3', 'Yes', 'No', 'Federal Territory of Kuala Lumpur', 'Multinational corp

In [49]:
df.head()

,Timestamp,Age,Race,Gender identity,Nationality,What is your highest education qualification?,Job title,Years of experience,How many jobs have you held in your career?,Are you working in/from Malaysia?,...,Industry_other,Job specialisation,Job specialisation_other,Average working hours per day,Average working days per week,"Monthly salary in Ringgit Malaysia (Gross: before EPF, tax deduction)",Starting salary for your first job in Ringgit Malaysia,How happy are you with your current salary/compensation package?,How satisfied are you with your current job overall?,Additional thoughts/insights you would like to share:
1,2022-03-10 12:51:52,27.0,Chinese,Male,Malaysia,Master's Degree (Overseas),Data Analyst,2,3,Yes,...,,,Data Science,8.0,5.0,6700,3500.0,4,3,
2,2022-03-10 12:51:57,23.0,Chinese,Male,Malaysia,Bachelor's Degree,IT specialist,1,1,Yes,...,,IT - Software,,10.0,5.0,5850,5400.0,4,2,High salary but have to deal with very toxic m...
3,2022-03-10 12:52:04,29.0,Chinese,Female,Malaysia,Bachelor's Degree (Overseas),Product specialist,6,4,Yes,...,,Project Management,,9.0,5.0,7500,2900.0,4,4,
4,2022-03-10 12:52:10,27.0,Chinese,Male,Malaysia,Bachelor's Degree,Senior executive - Data Analytics,3,2,Yes,...,,Other,Data Analytics,8.0,5.0,3800,2750.0,4,3,
5,2022-03-10 12:52:21,26.0,Chinese,Male,Malaysia,Bachelor's Degree,Demand Planner,4,3,No,...,Consumer Electronic,Other,Demand Planning / Commercial Operation,8.0,5.0,12000,3000.0,3,3,


In [ ]:
df.info()

In [51]:
df.to_csv('salary.csv')